Use of Dynamic Time Warping (DTW Algorithm) to compare two time series data, given that the movement of 2 stock price data do not occur simultaneously. 

Sources
- dtwParallel: A Python package to efficiently compute dynamic time warping between time series
https://www.sciencedirect.com/science/article/pii/S2352711023000602

- explanation : https://www.theaidream.com/post/dynamic-time-warping-dtw-algorithm-in-time-series

Explanation

The "corr_twoVar" utilizes the DTW Algorithm to identify similarity levels of a given metric


Alternatives
- If DTW is time-inefficient, FAST DTW (or) Sparce DTW can be used
- If 'similarity' is not sufficient, each stock being compared can be shifted by different time intervals, and then compared with standard PPMCC Corellations

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as pyplot
import seaborn as sbn
import matplotlib as mpl 


def corr_twoVarW():
